# Interface

This is what the interface looks like.
This notebook will have some code cells, but mostly pseudo-code to illustrate the interface.

## Importing modules

As any notebook we will start with importing the necessary modules.
**BUT** we will also import the eWaterCycle modules.

In [ ]:
# importing modules

import ewatercycle
import ewatercycle.model
import ewatercycle.forcing